In [ ]:
import re
import pandas as pd

In [ ]:
# language in ISO 639‑1 code
# both abstracts and short absctracts should be in this language
lang = 'en' # 'en', 'nl', 'fr'

# path for processing original data and saving dataframe in the data folder
abstracts_pathfile = '/full_abstracts/abstracts_' + lang + '.ttl' # check the name of file!
short_abstracts_pathfile = '/short_abstracts/short_abstracts_' + lang + '.ttl' # check the name of file!
output_path = '/data/data_' + lang + '.csv' # path for saving final dataframe

## Dataframe generation

In [ ]:
# reading short abstracts and urls for future concatenation
# download original files — https://databus.dbpedia.org/dbpedia/text/short-abstracts/

short_abstracts = []
short_urls = []

with open(short_abstracts_pathfile) as file:
    for string in file:
        try:
            string = string[len(' <http://nl.dbpedia.org/resource/')-1:]
            text = re.findall('.*> <http://www.w3.org/2000/01/rdf-schema#comment>', string)[0][:-len('> <http://www.w3.org/2000/01/rdf-schema#comment>')]
            short_urls.append(str(text))
            short_abstracts.append(re.findall('".*"', string)[0][1:-1])
        except:
            continue

if len(short_abstracts) == len(short_urls):
    print('Done! Number of abstracts is %i ans number of urls is %i'%(len(short_abstracts), len(short_urls)))

# dataframe with urls and short abstracts
data = {'url': short_urls, 'short_abstract': short_abstracts}
df_shorts = pd.DataFrame(data=data)

In [ ]:
# reading abstracts and urls for future concatenation
# download original files — http://downloads.dbpedia.org/2015-04/ext/nlp/abstracts/

abstracts = []
urls = []

with open(abstracts_pathfile) as file:
    for string in file:
        if 'nif:isString' in string:
            start = len('nif:isString """ ')
            finish = len('     ^^xsd:string')
            abstracts.append(string[start:-finish])
        
        if 'nif:sourceUrl' in string:
            start = len('nif:sourceUrl <http://nl.wikipedia.org/wiki/ ')
            finish = len('> . ')
            text = string[start:-finish]
            urls.append(str(text))

if len(abstracts) == len(urls):
    print('Done! Number of abstracts is %i ans number of urls is %i'%(len(abstracts), len(urls)))

# dataframe with urls and abstracts
data = {'url': urls, 'abstract': abstracts}
df_abstracts = pd.DataFrame(data=data)

In [ ]:
# merging of abstracts and short abstracts base on urls
final_df = df_abstracts.merge(df_shorts, on='url')

# saving dataframe in .csv
final_df.to_csv(output_path)